In [1]:
!pip install fasttext
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227137 sha256=101b3df7d6c01b953c9f5bd1de6b2597959c2fc81e8051762f8850477dc9595e
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [69]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import fasttext
from huggingface_hub import hf_hub_download
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [3]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is available


In [5]:
# 讀取.json
with open('./comments.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 提取 'comment' 與 'ad'
comments = [entry['comment'] for entry in data]
ads = [entry['ad'] for entry in data]

In [35]:
model_path = hf_hub_download(repo_id="facebook/fasttext-zh-vectors", filename="model.bin")
embad_model = fasttext.load_model(model_path)

In [64]:
# 將資料分成訓練集和測試集（4:1）
X_train, X_test, y_train, y_test = train_test_split(comments, ads, test_size=0.2, random_state=42, stratify=ads)

In [89]:
X_train_embed = [embad_model[x] for x in X_train]
X_test_embed = [embad_model[x] for x in X_test]

In [90]:
# 定義交叉驗證的折數
num_folds = 5
# 初始化 StratifiedKFold
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [91]:
# 創建空列表來保存每個折數的訓練和驗證分數
train_scores = []
val_scores = []

In [ ]:
for fold, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):
    print(f"Training on fold {fold + 1}/{num_folds}...")
    # 根據索引拆分訓練和驗證資料(from train data)
    trainX = np.array([X_train_embed[i] for i in train_index])
    valX = np.array([X_train_embed[i] for i in val_index])
    trainY = np.array([y_train[i] for i in train_index])
    valY = np.array([y_train[i] for i in val_index])
    # 計算類別權重
    train_class_weights = compute_class_weight('balanced', classes = np.unique(trainY), y = trainY)

    val_class_weights = compute_class_weight('balanced', classes = np.unique(valY), y = valY)
    val_sample_weights = np.zeros(len(valY))
    val_sample_weights[valY == 1] = val_class_weights[1]
    val_sample_weights[valY == 0] = val_class_weights[0]

    # 將類別權重轉換為字典格式
    train_class_weights = dict(enumerate(train_class_weights))

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])

    # 編譯模型
    model.compile(optimizer='adam',
                  loss=tf.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[keras.metrics.RecallAtPrecision(precision=0.8)])

    # 訓練模型
    history = model.fit(trainX,
                        trainY,
                        epochs=40,
                        batch_size=512,
                        verbose=0,
                        validation_data=(valX, valY, val_sample_weights),
                        class_weight=train_class_weights)

    # 評估模型
    train_score = model.evaluate(trainX, trainY, verbose=0)
    val_score = model.evaluate(valX, valY, verbose=0)

    train_scores.append(train_score)
    val_scores.append(val_score)


In [105]:
# 輸出每個折數的平均訓練和驗證分數
print("Average training scores:", np.mean(train_scores, axis=0))
print("Average validation scores:", np.mean(val_scores, axis=0))

Average training scores: [0.06774803 1.        ]
Average validation scores: [0.10432098 0.44225609]


In [106]:
y_pred = model.predict(np.array(X_test_embed), verbose=1)
y_pred = (y_pred > 0.5).astype(np.integer)

59/59 [==============================] - 0s 2ms/step


<ipython-input-106-fa0d75743cf9>:2: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  y_pred = (y_pred > 0.5).astype(np.integer)


In [107]:
y_test_np = np.array(y_test)
y_test_np = y_test_np[:, np.newaxis]
# accuracy = (y_pred == y_test_np).mean()
ad00 = np.logical_and(y_test_np == 0, y_pred == 0)
ad01 = np.logical_and(y_test_np == 0, y_pred == 1)
ad10 = np.logical_and(y_test_np == 1, y_pred == 0)
ad11 = np.logical_and(y_test_np == 1, y_pred == 1)
print("測試資料的數量:", len(y_test))
print("召回率:", sum(ad11) / (sum(ad10) + sum(ad11)))
print("本身不是廣告且判斷不是廣告的數量:", sum(ad00))
print("本身不是廣告且判斷是廣告的數量:", sum(ad01))
print("本身是廣告且判斷不是廣告的數量:", sum(ad10))
print("本身是廣告且判斷是廣告的數量:", sum(ad11))


測試資料的數量: 1867
召回率: [0.84313725]
本身不是廣告且判斷不是廣告的數量: [1791]
本身不是廣告且判斷是廣告的數量: [25]
本身是廣告且判斷不是廣告的數量: [8]
本身是廣告且判斷是廣告的數量: [43]


In [108]:
X_test_np = np.array(X_test)
X_test_np = X_test_np[:, np.newaxis]

In [109]:
ad_pred = X_test_np[y_pred == 1]
print("判斷為廣告的數量:", len(ad_pred))
print("判斷為廣告的句子:\n", ad_pred)

判斷為廣告的數量: 68
判斷為廣告的句子:
 ['讓我告訴你HBO277點c歐m上可以體驗一下無縣圓寶手遊，登入就享至尊會員，還有原創角色交易系統，玩膩了可以換現金。但自從最近更新了就不能進入遊戲。一開遊戲就閃退，帳號也一直不能成功梆定。希望有人可連絡我提供協助。人物和景色處理都獨樹一格，每-個地方都環環相扣，讓我體驗到遊戲設計師的創意小巧思，非常耐玩，而且劇情引人入勝，但是在敘事的時間也不會冗長，把握的剛剛好，跟我以前玩的手遊完全不一樣，是一個有層次跟故事的遊戲。'
 '請問一下現在是要維修到什麼時候會不會修太久了也沒有時間到底是要玩家等爽的喔'
 '我是一個熱愛天堂的小課玩家，本來對遊戲體驗雖然也有諸多小怨言，本著玩遊戲的心態也就得過且過，但是這一次 (亞特蘭提斯) 活動讓我感覺在 繼續課金就是白痴，遊戲體驗之糟糕 瞬間爆錶，為了一個解活動 任務 明知道會被殺，被嘲諷，還要像白痴一樣一直去，浪費一個小時，還是沒解到任務。讓人絕望的是就算我肯 繼續花錢依然無法改變這種糟糕的遊戲體驗。或許止損是我唯一的出路，尋找其他替代遊戲(╥﹏╥)'
 '超棒的說 活動填滿了小說和動漫的坑 ，我是在knn83.c om找手遊，裡面無需儲值就可以滿級會員和無限錢幣，進去就先送你神裝神獸，每週都還有很多活動禮包更新。還特地再做了動畫 遊戲體驗嗎 感覺出貨很容易的 至少比FGO好了 打的時候也很順手的 就是慢了點 如果週回速度可以再提高的話就更好 活動關卡有掉落物顯示就好 比起SAOMD 好很多 砸了不少錢下去 總括而言 真的挺好的 需要進步的位也有的 佢望能像FGO一樣 一路進步'
 '天堂m練妖精到50級大概要快一個月，用藍鑽買商品抽也抽不中，（不是商品嗎？為何用抽的）相賭博一樣，花了大概4萬台幣，（不是遊戲而已嗎？）聽說還有人花了80萬來抽，大概是有錢吧？！還跟遊戲公司打官司想把錢要回來，接果呢？這是很好玩的遊戲，別亂經營好嗎？！'
 '越改越爛 lag到不行 不得已重新下載 我4個帳號都手動 下載後竟然2個帳號沒辦法登入 認證失敗 認證失敗 如果還是這樣不能進入遊戲 我就刪除所以帳號退坑！'
 '美好的清晨來杯咖啡再配上破解手遊，你會發現生活就是如此簡單，先介紹9MAQ。?óＭ，上線滿V級版可以玩，資源元寶都是無限用的，上線就送無限貨幣，有點晚才發現。不管是場上的判斷還是選角

In [110]:
ad_test = X_test_np[np.logical_and(y_pred == 0, y_test_np == 1)]
print("應為廣告且判斷不是廣告的數量:", len(ad_test))
print("應為廣告且判斷不是廣告的句子:\n", ad_test)

應為廣告且判斷不是廣告的數量: 8
應為廣告且判斷不是廣告的句子:
 ['酣暢淋漓的華麗特技,更重要的是推薦去找蒼天牧師網玩鑽石無限滿V版,各種資源全都能無限這樣玩法太牛了,還有幾千個爆款無限版遊戲,前陣子好像有緊急維修,但事後沒有什麼補償給玩家感受上不是很好,但看在平時掉落那麼多獎勵的份上還是沒什麼特別不開心了,不過我還是建議下一次要補償一下,這樣才能夠更加留住玩家啊!這遊戲這麼好玩得讓更多人知道才行.'
 "聽覺上的享受'以後再也不會無聊。還有壹個特別的小技巧告訴大家。在網址ＨＣCD9.?OM可以學習如何創造以太幣的睡後收入.躺著就能賺到幣.我今天已經收了三w多超爽.後面甭看了.但不知道是不是之前有更新過.更新了目前最新版本以後.比上一個版本得還更容易卡住.然後就要用縮小畫面來重新更新.一個不小心就要重新啟動。他都不觉得烦"
 '哇塞～終於出了，遊戲等你來體驗，大家可以去SSr25.C〇м裡面許多破解版的遊戲，外加海量的資源等你來拿，這僅此福利的一小部分，我當初也不相信，自己去體驗後才驚訝是真的，不用再費勁心思的打怪升級穿裝備，讓你了解土豪是怎麼玩遊戲的。喜歡玩密室逃脫遊戲的玩家可以嘗試看看， 雖說劇情跟內容比較王道正統，但作為休閒娛樂也是有足夠的遊戲時數，對白有趣且配音完整，值得嘗試的好遊戲！整體完美，喜愛黑暗靈魂和巫師類型的必敗！另外非常建議用藍牙搖桿玩，一來螢幕玩久有點發燙，二來按鍵無反饋震動以致miss掉。除此之外一切完美！最後奉勸那位差評說操作要一個小時才能出招的玩家，你應該檢討的是自己的手機是不是太爛吧，給差評只會顯出你的無知。喜歡玩密室逃脫遊戲的玩家可以嘗試看看， 雖說劇情跟內容比較王道正統，但作為休閒娛樂也是有足夠的遊戲時數，對白有趣且配音完整，值得嘗試的好遊戲！'
 '非常好玩，跑酷一條命模式的Roguelike，關卡隨機，動作非常流暢，可說是完美移植。建議可以像UUtt77 .C(o)M的遊戲一樣都是登入享至尊會員和無限元寶的，這樣應該會更多人玩，且遊戲也會送鑽和送一些你要的，手游平台內一定有無限元寶版！沒玩過的新手記得每關結束後要把cells花光，不然死掉就沒了，可買的物品有些是一次性武器道具，這些先不要買浪費，買那些可以永久提升能力的，能力提升之後就比較容易過關。雖然遊戲價格貴了些但買了也不會後悔(相比之下惡果之地的價格簡直超超超佛心，強